# FDS Challenge: Starter Notebook

This notebook will guide you through the first steps of the competition. Our goal here is to show you how to:

1.  Load the `train.jsonl` and `test.jsonl` files from the competition data.
2.  Create a very simple set of features from the data.
3.  Train a basic model.
4.  Generate a `submission.csv` file in the correct format.
5.  Submit your results.

Let's get started!

In [57]:
from typing import Any
import json
import os
import re
from pprint import pprint
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from math import log2

from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_validate

pd.set_option("display.max_columns", 0)

### 1. Loading and Inspecting the Data

When you create a notebook within a Kaggle competition, the competition's data is automatically attached and available in the `../input/` directory.

The dataset is in a `.jsonl` format, which means each line is a separate JSON object. This is great because we can process it one line at a time without needing to load the entire large file into memory.

Let's write a simple loop to load the training data and inspect the first battle.

In [58]:
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.getcwd() #os.path.join('../input', COMPETITION_NAME)
UNTOUCHED = {'battle_id', 'player_won'}
INFORMATIVE = {
    "p1_unique_pokemon",
    "p2_unique_pokemon",
    "final_p1_hp",  
    "p1_fainted_count", 
    "p1_turns_statused", 
    "p1_missed_turns", 
    "p2_turns_statused",
    "p2_missed_turns", 
    "battle_id",
    "player_won"
}

train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r', encoding="utf-8") as f:
        train_data = [json.loads(line) for line in f]

    print(f"Successfully loaded {len(train_data)} battles.")

    #print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[0]
        
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])[:2] # Show first 2 turns
        
        #pprint(battle_for_display)
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")

except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from 'c:\Users\Stefano\Desktop\pokemon-challenge\train.jsonl'...
Successfully loaded 10000 battles.
    ...
    (battle_timeline has been truncated for display)


### 2. Basic Feature Engineering

A successful model will likely require creating many complex features. For this starter notebook, however, we will create a very simple feature set based **only on the initial team stats**. This will be enough to train a model and generate a submission file.

It's up to you to engineer more powerful features!

In [59]:
def features_check(data: dict) -> None:
    print("All battles have at least one turn: ", all(all(turn for turn in battle.get('battle_timeline', False)) for battle in data))
    print("All battles' turns have at least one P1 move: ", 
        all((
            any((turn.get("p1_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("All battles' turns have at least one P2 move: ", 
        all((
            any((turn.get("p2_move_details", False) for turn in battle.get('battle_timeline', False))) for battle in data
        ))
    )
    print("player_won feature always exists: ", all(('player_won' in battle for battle in data)))
    print("P1 Team always exists: ", all(battle.get('p1_team_details', False) for battle in data))
    print("P2 Team always exists: ", all(battle.get('p2_team_details', False) for battle in data))
    
    return None

In [60]:
def agg_pokemons_stats(prefix: str, stats: dict[str, Any]):
    return {
        f"{prefix}_mean_power": np.mean(stats["powers"]) if stats["powers"] else 0,
        f"{prefix}_mean_accuracy": np.mean(stats["accuracy"]) if stats["accuracy"] else 0,
        f"{prefix}_lost_hp": stats["lost_hp"],
        f"{prefix}_turns_statused": stats["turns_statused"],
        f"{prefix}_missed_turns": stats["missed_turns"],
        f"{prefix}_switches": stats["switches"],
        f"{prefix}_net_boost": stats["net_boost"],
    }

In [61]:
def _sanitize_name_for_feature(name: str) -> str:
    # lower, replace non-alnum with underscore, collapse multiple underscores
    s = re.sub(r'[^0-9a-zA-Z]+', '_', name.lower()).strip('_')
    return s or 'unknown'

In [62]:
def create_features(data: list[dict]) -> pd.DataFrame:
    # collect all unique pkm names across dataset (player1 team + p1 timeline entries)
    unique_p1_names = set()
    for battle in data:
        for p in battle.get('p1_team_details', []) or []:
            if p and 'name' in p:
                unique_p1_names.add(p['name'])
        for t in battle.get('battle_timeline', []) or []:
            p1_state = t.get('p1_pokemon_state', {})
            name = p1_state.get('name')
            if name:
                unique_p1_names.add(name)
    # map to sanitized feature names
    unique_name_to_feat = {name: f"p1_has_{_sanitize_name_for_feature(name)}" for name in unique_p1_names}

    feature_list = []
    
    features_check(data)

    for battle in data:
        features = {}

        # initialize all p1_has_<name> features to False for this battle
        for name, feat_key in unique_name_to_feat.items():
            features[feat_key] = False

        p1_stats = {
            "powers": [], 
            "accuracy": [], 
            "hp_t0": {}, 
            "lost_hp": 0, 
            "turns_statused": 0,
            "missed_turns": 0, 
            "priority": 0, 
            "switches": 0, 
            "net_boost": 0,
            "base_boosts": {"atk": 0, "def": 0, "spa": 0, "spd": 0, "spe": 0},
            "status_turns": {},
            "effect_turns": {},
            "hp_loss_while_statused": 0,
            "hp_loss_while_effect": 0,
        }
        
        p2_stats = {
            "powers": [], 
            "accuracy": [], 
            "hp_t0": {}, 
            "lost_hp": 0, 
            "turns_statused": 0,
            "missed_turns": 0, 
            "priority": 0, 
            "switches": 0, 
            "net_boost": 0,
            "base_boosts": {"atk": 0, "def": 0, "spa": 0, "spd": 0, "spe": 0},
            "status_turns": {},
            "effect_turns": {},
            "hp_loss_while_statused": 0,
            "hp_loss_while_effect": 0,
        }

        # --- Initial Pokémon states ---
        first_turn = battle["battle_timeline"][0] if battle.get("battle_timeline") else {}
        p1_lead = first_turn.get("p1_pokemon_state", {}).get("name", "")
        p2_lead = battle.get("p2_lead_details", {}).get("name", "")
        p1_prev_hp = first_turn.get("p1_pokemon_state", {}).get("hp_pct", 1.0)
        p2_prev_hp = first_turn.get("p2_pokemon_state", {}).get("hp_pct", 1.0)
        
        # --- Player 1 Team Features ---
        p1_team = battle.get('p1_team_details', None)
        features['p1_mean_hp'] = np.mean([p.get('base_hp') for p in p1_team])
        features['p1_mean_spe'] = np.mean([p.get('base_spe') for p in p1_team])
        features['p1_mean_atk'] = np.mean([p.get('base_atk') for p in p1_team])
        features['p1_mean_def'] = np.mean([p.get('base_def') for p in p1_team])
        features['p1_mean_special'] = np.mean([p.get('base_spa') for p in p1_team])

        # mark p1_has_<name> True for any pokemon in the team immediately
        if p1_team:
            for p in p1_team:
                if p and 'name' in p:
                    name = p['name']
                    if name in unique_name_to_feat:
                        features[unique_name_to_feat[name]] = True

        # --- Player 1 Lead Features ---
        p1_lead_stats = {}
        for pkm in p1_team:
            if pkm.get('name', '') == p1_lead:
                p1_lead_stats = {
                    'p1_lead_hp': pkm.get('base_hp', 0),
                    'p1_lead_atk': pkm.get('base_atk', 0),
                    'p1_lead_def': pkm.get('base_def', 0),
                    'p1_lead_special': pkm.get('base_spa', 0),
                    'p1_lead_spe': pkm.get('base_spe', 0),
                }
                break
        features.update(p1_lead_stats)

        # --- Player 2 Lead Features ---
        if p2_lead_details := battle.get('p2_lead_details'):
            features['p2_lead_hp'] = p2_lead_details.get('base_hp')
            features['p2_lead_spe'] = p2_lead_details.get('base_spe')
            features['p2_lead_atk'] = p2_lead_details.get('base_atk')
            features['p2_lead_def'] = p2_lead_details.get('base_def')
            features['p2_lead_special'] = p2_lead_details.get('base_spa')
            
            # Speed advantage
            features['spe_lead_adv'] = int(p1_lead_stats.get('p1_lead_spe', 0) > p2_lead_details.get('base_spe', 0))

        # --- Battle Timeline Features ---
        if timeline := battle.get('battle_timeline', []):
            p1_names = []
            p2_names = []
            p2_hp_deltas = []
            
            # For tracking HP loss under status/effects
            prev_p1_hp = None
            prev_p2_hp = None
            prev_p1_status = None
            prev_p2_status = None
            prev_p1_effects = set()
            prev_p2_effects = set()

            for t, t_stats in enumerate(timeline):
                p1_state = t_stats.get("p1_pokemon_state", {})
                p2_state = t_stats.get("p2_pokemon_state", {})

                p1_name = p1_state.get("name", "")
                p2_name = p2_state.get("name", "")
                p1_hp = p1_state.get("hp_pct", 1.0)
                p2_hp = p2_state.get("hp_pct", 1.0)
                p1_status = p1_state.get("status", "nostatus")
                p2_status = p2_state.get("status", "nostatus")
                p1_effects = p1_state.get("effects", ["noeffect"])
                p2_effects = p2_state.get("effects", ["noeffect"])

                # Track unique Pokémon
                if p1_name:
                    p1_names.append(p1_name)
                    # also mark the p1_has feature True if this name is in the global set
                    if p1_name in unique_name_to_feat:
                        features[unique_name_to_feat[p1_name]] = True
                if p2_name:
                    p2_names.append(p2_name)

                # --- Moves and accuracy ---
                for player, stats, move_key in [
                    ("p1", p1_stats, "p1_move_details"),
                    ("p2", p2_stats, "p2_move_details")
                ]:
                    move = t_stats.get(move_key)
                    if move:
                        stats["powers"].append(move.get("base_power", 0))
                        stats["accuracy"].append(move.get("accuracy", 0))
                    else:
                        stats["missed_turns"] += 1

                # --- Status and effect tracking ---
                if p1_status != "nostatus":
                    p1_stats["turns_statused"] += 1
                    p1_stats["status_turns"][p1_status] = p1_stats["status_turns"].get(p1_status, 0) + 1
                    
                if p2_status != "nostatus":
                    p2_stats["turns_statused"] += 1
                    p2_stats["status_turns"][p2_status] = p2_stats["status_turns"].get(p2_status, 0) + 1

                # Track effects
                for effect in p1_effects:
                    p1_stats["effect_turns"][effect] = p1_stats["effect_turns"].get(effect, 0) + 1
                for effect in p2_effects:
                    p2_stats["effect_turns"][effect] = p2_stats["effect_turns"].get(effect, 0) + 1
                
                # --- HP tracking ---
                # Track HP for each Pokémon
                if p1_name in p1_stats["hp_t0"]:
                    hp_delta = p1_stats["hp_t0"][p1_name] - p1_hp
                    if hp_delta > 0:
                        p1_stats["lost_hp"] += hp_delta
                p1_stats["hp_t0"][p1_name] = p1_hp
                
                if p2_name in p2_stats["hp_t0"]:
                    hp_delta = p2_stats["hp_t0"][p2_name] - p2_hp
                    if hp_delta > 0:
                        p2_stats["lost_hp"] += hp_delta
                p2_stats["hp_t0"][p2_name] = p2_hp

                # Track HP loss while statused or under effects
                p1_effects_set = set(p1_effects) if isinstance(p1_effects, list) else set()
                p2_effects_set = set(p2_effects) if isinstance(p2_effects, list) else set()
                
                if prev_p1_hp is not None and p1_hp < prev_p1_hp:
                    if prev_p1_status != "nostatus":
                        p1_stats["hp_loss_while_statused"] += prev_p1_hp - p1_hp
                    if len(prev_p1_effects) > 0:
                        p1_stats["hp_loss_while_effect"] += prev_p1_hp - p1_hp
                        
                if prev_p2_hp is not None and p2_hp < prev_p2_hp:
                    if prev_p2_status != "nostatus":
                        p2_stats["hp_loss_while_statused"] += prev_p2_hp - p2_hp
                    if len(prev_p2_effects) > 0:
                        p2_stats["hp_loss_while_effect"] += prev_p2_hp - p2_hp

                # Update previous values for next iteration
                prev_p1_hp = p1_hp
                prev_p2_hp = p2_hp
                prev_p1_status = p1_status
                prev_p2_status = p2_status
                prev_p1_effects = p1_effects_set
                prev_p2_effects = p2_effects_set

                # --- Switches ---
                if p1_name != p1_lead and p1_prev_hp > 0:
                    p1_stats["switches"] += 1

                if p2_name != p2_lead and p2_prev_hp > 0:
                    p2_stats["switches"] += 1

                # --- Boost tracking ---
                if p1_name != p1_lead:
                    p1_stats["base_boosts"] = {k: 0 for k in p1_stats["base_boosts"]}
                if p2_name != p2_lead:
                    p2_stats["base_boosts"] = {k: 0 for k in p2_stats["base_boosts"]}

                p1_boosts = p1_state.get("boosts", {})
                p2_boosts = p2_state.get("boosts", {})

                for stat in ["atk", "def", "spa", "spd", "spe"]:
                    p1_stats["net_boost"] += (p1_boosts.get(stat, 0) - p1_stats["base_boosts"].get(stat, 0))
                    p2_stats["net_boost"] += (p2_boosts.get(stat, 0) - p2_stats["base_boosts"].get(stat, 0))
                
                # Damage dealt calculation
                if t > 0:
                    prev_hp = timeline[t-1]['p2_pokemon_state']['hp_pct']
                    curr_hp = timeline[t]['p2_pokemon_state']['hp_pct']
                    p2_hp_deltas.append(prev_hp - curr_hp)

                # Update for next turn
                p1_lead = p1_name
                p2_lead = p2_name
                p1_prev_hp = p1_hp
                p2_prev_hp = p2_hp
                p1_stats["base_boosts"] = p1_boosts.copy()
                p2_stats["base_boosts"] = p2_boosts.copy()

            # Aggregate features
            features['p1_unique_pokemon'] = len(set(p1_names))
            features['p2_unique_pokemon'] = len(set(p2_names))
            features['n_turns'] = len(timeline)
            features['mean_damage_dealt'] = np.mean([d for d in p2_hp_deltas if d > 0]) if p2_hp_deltas else None
            
            # Final HP and KO counts
            last_state = timeline[-1]['p1_pokemon_state']
            features['final_p1_hp'] = last_state.get('hp_pct', None)
            features['p1_fainted_count'] = sum(t['p1_pokemon_state']['status'] == 'fnt' for t in timeline)
            features['p2_fainted_count'] = sum(t['p2_pokemon_state']['status'] == 'fnt' for t in timeline)
            
            # --- New HP-based features --- (unchanged)
            p1_final_hp = np.mean(list(p1_stats["hp_t0"].values())) if p1_stats["hp_t0"] else 1.0
            p2_final_hp = np.mean(list(p2_stats["hp_t0"].values())) if p2_stats["hp_t0"] else 1.0
            features['p1_mean_final_hp'] = p1_final_hp
            features['p2_mean_final_hp'] = p2_final_hp
            features['p1_hp_loss_ratio'] = p1_stats["lost_hp"] / (p1_stats["lost_hp"] + p1_final_hp + 1e-6)
            features['p2_hp_loss_ratio'] = p2_stats["lost_hp"] / (p2_stats["lost_hp"] + p2_final_hp + 1e-6)
            features['hp_diff_lost'] = p2_stats["lost_hp"] - p1_stats["lost_hp"]
            features['hp_diff_final'] = p1_final_hp - p2_final_hp
            features['hp_ratio_p1_p2'] = p1_stats["lost_hp"] / (p2_stats["lost_hp"] + 1e-6)
            
            features['p1_mean_hp_pct'] = np.mean([v for v in p1_stats["hp_t0"].values() if v > 0]) if p1_stats["hp_t0"] else 1.0
            features['p2_mean_hp_pct'] = np.mean([v for v in p2_stats["hp_t0"].values() if v > 0]) if p2_stats["hp_t0"] else 1.0
            features['p1_min_hp'] = np.min(list(p1_stats["hp_t0"].values())) if p1_stats["hp_t0"] else 1.0
            features['p2_min_hp'] = np.min(list(p2_stats["hp_t0"].values())) if p2_stats["hp_t0"] else 1.0
            
            # Survival ratios
            p1_alive = sum(1 for v in p1_stats["hp_t0"].values() if v > 0)
            p2_alive = sum(1 for v in p2_stats["hp_t0"].values() if v > 0)
            features['p1_survival_ratio'] = p1_alive / (len(p1_stats["hp_t0"]) + 1e-6)
            features['p2_survival_ratio'] = p2_alive / (len(p2_stats["hp_t0"]) + 1e-6)
            features['survival_diff'] = features['p1_survival_ratio'] - features['p2_survival_ratio']
            
            # HP loss per Pokémon and per turn
            features['p1_avg_hp_lost_per_pkm'] = p1_stats["lost_hp"] / (len(p1_stats["hp_t0"]) + 1e-6)
            features['p2_avg_hp_lost_per_pkm'] = p2_stats["lost_hp"] / (len(p2_stats["hp_t0"]) + 1e-6)
            features['p1_hp_loss_per_turn'] = p1_stats["lost_hp"] / (features['n_turns'] + 1e-6)
            features['p2_hp_loss_per_turn'] = p2_stats["lost_hp"] / (features['n_turns'] + 1e-6)
            features['hp_delta_per_turn'] = (p2_stats["lost_hp"] - p1_stats["lost_hp"]) / (features['n_turns'] + 1e-6)
            features['hp_relative_strength'] = (p1_final_hp - p2_final_hp) / (abs(p1_final_hp) + abs(p2_final_hp) + 1e-6)
            
            # --- Status and effect impact features ---
            features["p1_hp_loss_status_ratio"] = p1_stats["hp_loss_while_statused"] / (p1_stats["lost_hp"] + 1e-6)
            features["p2_hp_loss_status_ratio"] = p2_stats["hp_loss_while_statused"] / (p2_stats["lost_hp"] + 1e-6)
            features["p1_hp_loss_effect_ratio"] = p1_stats["hp_loss_while_effect"] / (p1_stats["lost_hp"] + 1e-6)
            features["p2_hp_loss_effect_ratio"] = p2_stats["hp_loss_while_effect"] / (p2_stats["lost_hp"] + 1e-6)
            features["hp_diff_statused"] = p2_stats["hp_loss_while_statused"] - p1_stats["hp_loss_while_statused"]
            features["hp_diff_effected"] = p2_stats["hp_loss_while_effect"] - p1_stats["hp_loss_while_effect"]
            features["p1_resistance_index"] = 1 - features["p1_hp_loss_status_ratio"]
            features["p2_resistance_index"] = 1 - features["p2_hp_loss_status_ratio"]
            features["resistance_diff"] = features["p1_resistance_index"] - features["p2_resistance_index"]
            
            # Add status and effect turn counts
            for status, count in p1_stats["status_turns"].items():
                features[f"p1_pkm_{status}"] = count
            for status, count in p2_stats["status_turns"].items():
                features[f"p2_pkm_{status}"] = count
            for effect, count in p1_stats["effect_turns"].items():
                features[f"p1_pkm_{effect}"] = count
            for effect, count in p2_stats["effect_turns"].items():
                features[f"p2_pkm_{effect}"] = count
            
        else:
            features.update({
                'p1_unique_pokemon': None,
                'p2_unique_pokemon': None,
                'n_turns': 0,
                'mean_damage_dealt': None,
                'final_p1_hp': None,
                'p1_fainted_count': None,
                'p2_fainted_count': None,
            })

        features.update(agg_pokemons_stats("p1", p1_stats))
        features.update(agg_pokemons_stats("p2", p2_stats))

        features['battle_id'] = battle.get('battle_id')
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])
            
        feature_list.append(features)
        
    return pd.DataFrame(feature_list)

In [63]:
print("Processing training data...")
train_df = create_features(train_data)

print("\nProcessing test data...")
with open(test_file_path, 'r', encoding="utf-8") as f:
    test_data = [json.loads(line) for line in f]
        
test_df = create_features(test_data)

Processing training data...
All battles have at least one turn:  True
All battles' turns have at least one P1 move:  False
All battles' turns have at least one P2 move:  False
player_won feature always exists:  True
P1 Team always exists:  True
P2 Team always exists:  False


c:\Users\Stefano\anaconda3\envs\sapienza\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Stefano\anaconda3\envs\sapienza\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Processing test data...
All battles have at least one turn:  True
All battles' turns have at least one P1 move:  False
All battles' turns have at least one P2 move:  False
player_won feature always exists:  False
P1 Team always exists:  True
P2 Team always exists:  False


In [64]:
#train_df.corr()

In [65]:
keepers = list(set(train_df.columns).difference(UNTOUCHED))

In [66]:
scaler = StandardScaler(with_mean=True, with_std=True)

train_df[keepers] = scaler.fit_transform(train_df[keepers])
test_df[keepers] = scaler.fit_transform(test_df[keepers])

In [67]:
print("\nTraining dataset preview:")
display(train_df.head())
display(train_df.describe())
display(train_df.dtypes)

print("\nTesting dataset preview:")
display(test_df.head())
display(test_df.describe())
display(test_df.dtypes)


Training dataset preview:


,p1_has_zapdos,p1_has_persian,p1_has_gengar,p1_has_rhydon,p1_has_dragonite,p1_has_cloyster,p1_has_lapras,p1_has_jynx,p1_has_victreebel,p1_has_snorlax,p1_has_exeggutor,p1_has_golem,p1_has_slowbro,p1_has_alakazam,p1_has_articuno,p1_has_tauros,p1_has_jolteon,p1_has_chansey,p1_has_starmie,p1_has_charizard,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p2_pkm_par,p1_pkm_noeffect,p1_pkm_reflect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won,p2_pkm_substitute,p2_pkm_reflect,p1_pkm_frz,p1_pkm_wrap,p1_pkm_tox,p1_pkm_psn,p1_pkm_confusion,p1_pkm_substitute,p2_pkm_confusion,p1_pkm_clamp,p2_pkm_clamp,p2_pkm_tox,p2_pkm_psn,p2_pkm_brn,p2_pkm_wrap,p1_pkm_brn,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_firespin,p2_pkm_firespin
0,-0.603074,-0.187036,-0.576273,-0.595522,-0.199834,-0.402467,-0.341606,-0.559949,-0.202524,0.445173,0.573809,-0.371718,-0.343246,1.051737,-0.193247,0.341423,-0.317357,0.42403,1.082819,-0.09259,0.202093,0.520813,-0.732064,-0.745443,0.644639,-0.508230,0.553031,1.039088,-0.562587,0.763073,-0.279683,0.490498,0.642099,1.185993,-0.741369,-0.747885,-1.404495,-1.331840,0.0,-0.307048,...,-0.956921,0.069635,-1.097798,0.533849,-0.118412,-0.823055,-0.564623,-1.089872,-1.109216,-0.385666,0.048317,0.446467,0.785598,0.029295,1.042992,2.443488,0.629866,-1.119474,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.603074,-0.187036,-0.576273,-0.595522,-0.199834,-0.402467,-0.341606,1.785878,-0.202524,0.445173,0.573809,-0.371718,2.913360,-0.950808,-0.193247,0.341423,-0.317357,0.42403,-0.923516,-0.09259,0.761596,-1.738011,-0.732064,-0.492591,-0.759727,-0.429812,-0.327042,-0.661341,-0.806141,0.078764,-0.450188,0.723525,-0.711192,-0.611967,1.072092,-0.747885,0.918520,0.950271,0.0,-1.387738,...,-1.349168,0.550447,NaN,0.533849,1.450473,0.172684,-0.085086,-0.341616,0.192108,0.302593,0.048317,0.170319,0.322559,0.539208,-1.090316,0.158909,0.629866,-0.554240,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.603074,-0.187036,1.735290,1.679199,-0.199834,-0.402467,-0.341606,-0.559949,-0.202524,0.445173,0.573809,-0.371718,-0.343246,-0.950808,-0.193247,0.341423,-0.317357,0.42403,-0.923516,-0.09259,0.823762,-1.224642,0.906915,0.097399,-0.759727,0.040699,1.257090,1.039088,0.655186,-1.289855,6.199509,-2.538851,-3.147117,-2.409928,-0.482303,1.337103,-2.566002,-1.331840,0.0,-0.658753,...,-0.956921,-1.613204,0.438778,-2.347998,-1.285978,-0.326265,-1.257942,0.406640,-1.109216,-0.729796,0.048317,-0.309510,-0.358604,-0.268309,0.509665,0.485277,-1.756120,-0.554240,2,1,3.504749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.658171,-0.187036,1.735290,-0.595522,-0.199834,-0.402467,-0.341606,-0.559949,-0.202524,0.445173,0.573809,-0.371718,-0.343246,-0.950808,-0.193247,0.341423,-0.317357,0.42403,-0.923516,-0.09259,0.637262,0.007444,-0.029644,-0.492591,1.112761,-0.508230,0.201002,0.188873,0.898740,0.591996,0.231832,0.257471,1.995390,1.635483,-2.295764,-0.747885,-0.242987,-1.331840,0.0,0.159132,...,NaN,0.550447,NaN,-0.426767,0.404353,-0.060591,0.337487,1.341960,0.192108,-0.729796,0.048317,1.549122,-0.440752,0.709810,-1.090316,-0.493828,-0.733554,0.010994,3,1,NaN,-0.646458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.603074,-0.187036,-0.576273,-0.595522,-0.199834,2.484673,-0.341606,-0.559949,-0.202524,0.445173,0.573809,-0.371718,-0.343246,1.051737,-0.193247,0.341423,-0.317357,0.42403,-0.923516,-0.09259,0.077760,-0.403251,-0.263784,0.855957,0.293548,-0.586649,-0.327042,-0.321255,1.142295,0.934151,-0.279683,0.490498,0.642099,1.185

,p1_has_zapdos,p1_has_persian,p1_has_gengar,p1_has_rhydon,p1_has_dragonite,p1_has_cloyster,p1_has_lapras,p1_has_jynx,p1_has_victreebel,p1_has_snorlax,p1_has_exeggutor,p1_has_golem,p1_has_slowbro,p1_has_alakazam,p1_has_articuno,p1_has_tauros,p1_has_jolteon,p1_has_chansey,p1_has_starmie,p1_has_charizard,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p2_pkm_par,p1_pkm_noeffect,p1_pkm_reflect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,player_won,p2_pkm_substitute,p2_pkm_reflect,p1_pkm_frz,p1_pkm_wrap,p1_pkm_tox,p1_pkm_psn,p1_pkm_confusion,p1_pkm_substitute,p2_pkm_confusion,p1_pkm_clamp,p2_pkm_clamp,p2_pkm_tox,p2_pkm_psn,p2_pkm_brn,p2_pkm_wrap,p1_pkm_brn,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_firespin,p2_pkm_firespin
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,9.987000e+03,...,8.574000e+03,9.998000e+03,2704.000000,9.998000e+03,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.00000,10000.000000,5.040000e+02,2.571000e+03,1.505000e+03,3.100000e+01,1.710000e+02,8.200000e+01,4.070000e+02,4.990000e+02,4.010000e+02,4.200000e+01,4.400000e+01,1.750000e+02,9.600000e+01,8.100000e+01,3.000000e+01,9.000000e+01,2.0,2.0,2.000000,2.000000
mean,-1.421085e-18,4.831691e-17,3.090861e-17,1.016076e-16,-7.105427e-17,4.121148e-17,4.973799e-18,6.394885e-18,-3.126388e-17,-4.405365e-17,9.805490e-17,4.973799e-17,4.831691e-17,-7.105427e-18,1.278977e-17,1.875833e-16,7.673862e-17,-7.531753e-17,0.000000,-1.421085e-17,-4.185097e-16,-9.471535e-16,7.389644e-17,3.140599e-16,1.675460e-15,-3.765876e-17,3.268497e-17,-3.268497e-17,2.422951e-16,1.854517e-16,5.115908e-17,1.605827e-16,1.012523e-16,7.105427e-19,2.785328e-16,-3.126388e-17,-1.136868e-16,-2.842171e-16,0.0,2.443891e-16,...,-6.961230e-17,-4.292537e-16,0.000000,1.129989e-16,-1.548983e-16,1.150369e-15,-2.160050e-16,1.136868e-17,-1.364242e-16,-7.531753e-17,2.149392e-17,1.620037e-16,-1.056577e-15,-4.547474e-17,-9.094947e-17,1.364242e-16,-2.614797e-16,-2.842171e-17,4999.50000,0.500000,4.934325e-17,4.974628e-17,-6.845761e-17,-2.865092e-17,-3.635818e-17,8.665155e-17,-6.983221e-17,1.174745e-16,-8.859635e-17,7.665826e-17,-2.018587e-17,-4.567775e-17,-8.789266e-17,-6.579099e-17,-2.220446e-17,-6.908054e-17,0.0,0.0,0.000000,0.000000
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,1.000050e+00,...,1.000058e+00,1.000050e+00,1.000185,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,2886.89568,0.500025,1.000994e+00,1.000195e+00,1.000332e+00,1.016530e+0

p1_has_zapdos        float64
p1_has_persian       float64
p1_has_gengar        float64
p1_has_rhydon        float64
p1_has_dragonite     float64
                      ...   
p1_pkm_brn           float64
p1_pkm_typechange    float64
p2_pkm_typechange    float64
p1_pkm_firespin      float64
p2_pkm_firespin      float64
Length: 118, dtype: object


Testing dataset preview:


,p1_has_zapdos,p1_has_persian,p1_has_gengar,p1_has_rhydon,p1_has_dragonite,p1_has_cloyster,p1_has_lapras,p1_has_jynx,p1_has_victreebel,p1_has_snorlax,p1_has_exeggutor,p1_has_golem,p1_has_slowbro,p1_has_alakazam,p1_has_articuno,p1_has_tauros,p1_has_jolteon,p1_has_chansey,p1_has_starmie,p1_has_charizard,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p1_pkm_noeffect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,p2_pkm_frz,p1_pkm_wrap,p2_pkm_clamp,p2_pkm_wrap,p1_pkm_brn,p1_pkm_confusion,p2_pkm_fnt,p1_pkm_clamp,p1_pkm_tox,p1_pkm_psn,p2_pkm_reflect,p1_pkm_reflect,p2_pkm_substitute,p2_pkm_tox,p2_pkm_brn,p2_pkm_confusion,p2_pkm_psn,p1_pkm_substitute,p1_pkm_firespin,p2_pkm_firespin,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_disable
0,-0.601687,5.101669,1.687932,-0.590593,-0.204655,-0.402132,-0.348682,-0.541596,-0.192689,0.459609,0.558562,-0.362245,-0.354428,-0.95616,-0.202524,0.353354,-0.316784,0.438707,-0.936041,-0.101514,0.347635,0.297212,-0.497947,-0.937347,-0.327487,2.427470,-1.903090,-1.690965,-0.344096,-1.460522,-0.111797,1.206605,0.120196,0.067841,-0.216179,-0.763728,-0.252920,-0.210231,0.0,0.358300,...,0.545824,0.522078,-0.069245,0.803228,0.429182,1.155070,1.166160,-0.732493,0.021715,0.672315,-0.350403,0.325505,1.010900,-0.172866,-0.377371,2.388154,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.601687,-0.196014,-0.592441,-0.590593,4.886269,2.486746,-0.348682,-0.541596,5.189698,-2.175763,-1.790311,-0.362245,-0.354428,-0.95616,-0.202524,0.353354,3.156725,-2.279427,1.068329,-0.101514,-3.135315,2.448247,2.499830,2.635469,-0.209235,-0.435025,0.209725,0.193802,-0.096794,1.272235,-0.437793,0.730007,-0.697689,-0.607419,1.098777,1.309367,-1.414171,0.926151,0.0,-1.759427,...,0.545824,0.522078,-0.312162,-2.037690,-1.858883,-1.600432,-1.116337,-1.424372,1.161598,1.285556,1.153320,-0.262702,0.313102,3.749966,-0.036476,0.026217,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.601687,-0.196014,-0.592441,1.693215,-0.204655,2.486746,-0.348682,-0.541596,5.189698,0.459609,-1.790311,-0.362245,-0.354428,-0.95616,-0.202524,0.353354,-0.316784,0.438707,-0.936041,-0.101514,0.531594,-1.751393,1.825911,1.784799,-2.456024,2.427470,-1.903090,-1.690965,-0.344096,-1.460522,-0.437793,0.730007,-0.697689,-0.607419,1.098777,-0.763728,-0.252920,0.926151,0.0,-2.361742,...,0.072360,-1.656039,-1.296566,-3.555448,-1.430696,-2.151533,-0.464195,-1.078432,0.021715,-0.480580,-0.290254,-1.185299,1.010900,-0.499769,-0.377371,0.616702,2,0.052652,-0.940971,-1.027358,0.279926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.601687,-0.196014,-0.592441,-0.590593,-0.204655,2.486746,-0.348682,1.846394,-0.192689,0.459609,0.558562,-0.362245,-0.354428,-0.95616,-0.202524,-2.830019,3.156725,0.438707,-0.936041,-0.101514,0.102357,-0.522230,-1.078912,0.083458,0.263773,-0.435025,-0.318479,-0.662910,-0.838700,0.076654,2.985166,-3.559383,2.573852,0.292928,-2.583099,1.309367,-2.575421,-0.210231,0.0,0.456963,...,-0.164372,0.522078,-1.357988,0.803228,0.224281,1.155070,-1.442408,-0.732493,-0.548227,0.683265,0.707772,0.256305,1.010900,-0.172866,-1.400056,-1.154751,3,NaN,NaN,NaN,NaN,3.57628,-0.680169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.601687,-0.196014,1.687932,-0.590593,-0.204655,-0.402132,-0.348682,-0.541596,-0.192689,0.459609,0.558562,-0.362245,-0.354428,-0.95616,-0.202524,0.353354,-0.316784,0.438707,1.068329,-0.101514,0.286316,0.297212,-0.381754,-0.512012,0.500277,-0.512390,0.561861,1.050514,-0.591398,0.759843,-0.274795,0.253408,

,p1_has_zapdos,p1_has_persian,p1_has_gengar,p1_has_rhydon,p1_has_dragonite,p1_has_cloyster,p1_has_lapras,p1_has_jynx,p1_has_victreebel,p1_has_snorlax,p1_has_exeggutor,p1_has_golem,p1_has_slowbro,p1_has_alakazam,p1_has_articuno,p1_has_tauros,p1_has_jolteon,p1_has_chansey,p1_has_starmie,p1_has_charizard,p1_mean_hp,p1_mean_spe,p1_mean_atk,p1_mean_def,p1_mean_special,p1_lead_hp,p1_lead_atk,p1_lead_def,p1_lead_special,p1_lead_spe,p2_lead_hp,p2_lead_spe,p2_lead_atk,p2_lead_def,p2_lead_special,spe_lead_adv,p1_unique_pokemon,p2_unique_pokemon,n_turns,mean_damage_dealt,...,p1_pkm_noeffect,p2_pkm_noeffect,p1_mean_power,p1_mean_accuracy,p1_lost_hp,p1_turns_statused,p1_missed_turns,p1_switches,p1_net_boost,p2_mean_power,p2_mean_accuracy,p2_lost_hp,p2_turns_statused,p2_missed_turns,p2_switches,p2_net_boost,battle_id,p2_pkm_frz,p1_pkm_wrap,p2_pkm_clamp,p2_pkm_wrap,p1_pkm_brn,p1_pkm_confusion,p2_pkm_fnt,p1_pkm_clamp,p1_pkm_tox,p1_pkm_psn,p2_pkm_reflect,p1_pkm_reflect,p2_pkm_substitute,p2_pkm_tox,p2_pkm_brn,p2_pkm_confusion,p2_pkm_psn,p1_pkm_substitute,p1_pkm_firespin,p2_pkm_firespin,p1_pkm_typechange,p2_pkm_typechange,p1_pkm_disable
count,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.0,4.998000e+03,...,4.999000e+03,4.999000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5.000000e+03,5000.000000,7.440000e+02,2.100000e+01,1.900000e+01,1.600000e+01,3.400000e+01,2.080000e+02,1.484000e+03,15.000000,8.500000e+01,4.700000e+01,1.253000e+03,1.333000e+03,2.470000e+02,9.100000e+01,3.900000e+01,2.270000e+02,3.700000e+01,2.390000e+02,1.0,1.0,1.0,1.0,1.0
mean,1.136868e-17,4.405365e-17,4.263256e-18,2.806644e-17,3.126388e-17,-8.526513e-17,-2.771117e-17,-3.126388e-17,-5.684342e-17,-2.273737e-17,2.842171e-17,-8.526513e-17,0.000000,-5.684342e-17,1.705303e-17,-1.250555e-16,-5.684342e-18,1.847411e-17,-6.821210e-17,1.705303e-17,-9.123369e-16,1.355005e-15,3.637979e-16,-7.560175e-16,-1.540457e-15,-8.526513e-17,1.421085e-16,-9.947598e-18,3.396394e-16,-2.415845e-17,-4.831691e-17,-3.467449e-16,-2.273737e-17,-9.166001e-17,1.634248e-17,-3.126388e-17,-5.002221e-16,4.774847e-16,0.0,1.339909e-16,...,-3.581852e-16,3.752416e-16,3.836931e-17,1.720935e-15,4.547474e-16,1.364242e-16,-1.818989e-16,1.136868e-17,3.979039e-17,3.581135e-16,-1.016076e-15,9.094947e-17,4.547474e-17,-1.136868e-16,2.273737e-16,-1.136868e-17,2499.500000,-2.387576e-17,3.172066e-17,-1.402387e-16,-1.387779e-17,-1.306145e-16,-5.764620e-17,-2.609473e-16,0.000000,-8.359326e-17,-8.031401e-17,-2.268293e-17,8.528645e-17,-2.876691e-17,5.368111e-17,-3.985416e-17,-3.130144e-17,1.200241e-17,2.972982e-17,0.0,0.0,0.0,0.0,6.0
std,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,0.0,1.000100e+00,...,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1.000100e+00,1443.520003,1.000673e+00,1.024695e+00,1.027402e+00,1.032796e+00,1.015038e+00,1.002413e+00,1.000337e+00,1.035098,1.005935e+

p1_has_zapdos        float64
p1_has_persian       float64
p1_has_gengar        float64
p1_has_rhydon        float64
p1_has_dragonite     float64
                      ...   
p1_pkm_firespin      float64
p2_pkm_firespin      float64
p1_pkm_typechange    float64
p2_pkm_typechange    float64
p1_pkm_disable       float64
Length: 118, dtype: object

### 3. Training Models

In [68]:
# Define predictor features (X) and target (y)
X_train = train_df[keepers] #[keepers]
print(len(train_df.columns))
y_train = train_df['player_won']

X_test = test_df[keepers] #[keepers]

print("Training...")
model = XGBClassifier(
    random_state=100,
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    eval_metric='logloss',
    n_jobs=-1
)
model = LGBMClassifier(
    learning_rate=0.03165579861591166, 
    num_leaves=125,
    max_depth=3, 
    min_child_samples=29, 
    reg_alpha=7.1991672853117725,  
    reg_lambda=3.052954302286083, 
    feature_fraction=0.9504554803940148, 
    bagging_fraction=0.9731871954622454, 
    bagging_freq=6, 
    objective='binary',
    n_estimators=335,
    random_state=100
)
model.fit(X_train, y_train)
print("Model training complete.")

118
Training...
[LightGBM] [Warning] feature_fraction is set=0.9504554803940148, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9504554803940148
[LightGBM] [Warning] bagging_fraction is set=0.9731871954622454, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9731871954622454
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.9504554803940148, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9504554803940148
[LightGBM] [Warning] bagging_fraction is set=0.9731871954622454, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9731871954622454
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Info] Number of positive: 5000, number of negative: 5000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002815 se

In [69]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=100)
cv_results = cross_validate(
    model,
    X_train,
    y_train,
    cv=cv,
    scoring={
        "accuracy_score": make_scorer(accuracy_score),
        "precision_score": make_scorer(precision_score),
        "recall_score": make_scorer(recall_score),
        "f1_score": make_scorer(f1_score),
        "roc_auc_score": make_scorer(roc_auc_score)
    },
    return_train_score=True,
    n_jobs=-1
)

results_df = pd.DataFrame(cv_results)
display(results_df)

results_df = pd.DataFrame(cv_results)
summary = results_df.filter(regex='(train_|test_)').describe().loc[['mean', 'std']].T
summary.rename(columns={'mean': 'Mean', 'std': 'Std'}, inplace=True)
display(summary)

,fit_time,score_time,test_accuracy_score,train_accuracy_score,test_precision_score,train_precision_score,test_recall_score,train_recall_score,test_f1_score,train_f1_score,test_roc_auc_score,train_roc_auc_score
0,3.389078,0.018001,0.825,0.853556,0.826962,0.856886,0.822,0.848889,0.824473,0.852869,0.825,0.853556
1,3.585176,0.026000,0.826,0.851222,0.831301,0.854452,0.818,0.846667,0.824597,0.850541,0.826,0.851222
2,3.467098,0.023518,0.829,0.852111,0.833671,0.854871,0.822,0.848222,0.827795,0.851534,0.829,0.852111
3,3.134910,0.022512,0.825,0.851556,0.818004,0.853599,0.836,0.848667,0.826904,0.851125,0.825,0.851556
4,3.427098,0.021001,0.834,0.851333,0.828740,0.854325,0.842,0.847111,0.835317,0.850703,0.834,0.851333
5,3.100394,0.019023,0.847,0.850889,0.862213,0.853719,0.826,0.846889,0.843718,0.850290,0.847,0.850889
6,3.406100,0.023001,0.842,0.851444,0.847561,0.852620,0.834,0.849778,0.840726,0.851196,0.842,0.851444
7,3.601173,0.020023,0.854,0.848222,0.865702,0.850559,0.838,0.844889,0.851626,0.847715,0.854,0.848222
8,2.992885,0.020511,0.844,0.850222,0.848178,0.851629,0.838,0.848222,0.843058,0.849922,0.844,0.850222
9,3.001375,0.024022,0.847,0.849667,0.844930,0.851777,0.850,0.846667,0.847458,0.849214,0.847,0.849667


,Mean,Std
test_accuracy_score,0.837300,0.010771
train_accuracy_score,0.851022,0.001435
test_precision_score,0.840726,0.015611
train_precision_score,0.853444,0.001852
test_recall_score,0.832600,0.010244
train_recall_score,0.847600,0.001420
test_f1_score,0.836567,0.010096
train_f1_score,0.850511,0.001390
test_roc_auc_score,0.837300,0.010771
train_roc_auc_score,0.851022,0.001435


### 4. Creating the Submission File

The competition requires a `.csv` file with two columns: `battle_id` and `player_won`. Let's use our trained model to make predictions on the test set and format them correctly.

In [70]:
print("Generating predictions on the test set...")
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    'player_won': model.predict(X_test)
})

submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' file created successfully!")
display(submission_df.head())

Generating predictions on the test set...
[LightGBM] [Warning] feature_fraction is set=0.9504554803940148, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9504554803940148
[LightGBM] [Warning] bagging_fraction is set=0.9731871954622454, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9731871954622454
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6

'submission.csv' file created successfully!


,battle_id,player_won
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


### 5. Submitting Your Results

Once you have generated your `submission.csv` file, there are two primary ways to submit it to the competition.

---

#### Method A: Submitting Directly from the Notebook

This is the standard method for code competitions. It ensures that your submission is linked to the code that produced it, which is crucial for reproducibility.

1.  **Save Your Work:** Click the **"Save Version"** button in the top-right corner of the notebook editor.
2.  **Run the Notebook:** In the pop-up window, select **"Save & Run All (Commit)"** and then click the **"Save"** button. This will run your entire notebook from top to bottom and save the output, including your `submission.csv` file.
3.  **Go to the Viewer:** Once the save process is complete, navigate to the notebook viewer page. 
4.  **Submit to Competition:** In the viewer, find the **"Submit to Competition"** section. This is usually located in the header of the output section or in the vertical "..." menu on the right side of the page. Clicking the **Submit** button this will submit your generated `submission.csv` file.

After submitting, you will see your score in the **"Submit to Competition"** section or in the [Public Leaderboard](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?).

---

#### Method B: Manual Upload

You can also generate your predictions and submission file using any environment you prefer (this notebook, Google Colab, or your local machine).

1.  **Generate the `submission.csv` file** using your model.
2.  **Download the file** to your computer.
3.  **Navigate to the [Leaderboard Page](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?)** and click on the **"Submit Predictions"** button.
4.  **Upload Your File:** Drag and drop or select your `submission.csv` file to upload it.

This method is quick, but keep in mind that for the final evaluation, you might be required to provide the code that generated your submission.

Good luck!